This notebook applies regression to the hard drive data and tries to predict the number of days it takes for a hard drive to fail

In [1]:
import pandas as pd
import numpy as np
import random

In [2]:
hard_drive_df = pd.read_csv('failed_hard_drive_df.csv')

In [3]:
hard_drive_df.date = pd.to_datetime(hard_drive_df.date)

In [4]:
hard_drive_df.columns

Index(['date', 'serial_number', 'model', 'failure', 'smart_184_raw',
       'smart_242_raw', 'smart_241_raw', 'smart_194_raw', 'smart_193_raw',
       'smart_198_raw',
       ...
       'smart_194_from_mean_30', 'smart_193_from_mean_30',
       'smart_198_from_mean_30', 'smart_197_from_mean_30',
       'smart_188_from_mean_30', 'smart_187_from_mean_30',
       'smart_5_from_mean_30', 'failure_date', 'remaining_useful_life',
       'RUL_30'],
      dtype='object', length=117)

In [5]:
hard_drive_df = hard_drive_df[['date','remaining_useful_life',
       'RUL_30', 'failure', 'model',
       'serial_number','smart_187_from_mean_10', 'smart_187_raw', 'smart_187_rw10_mean',
       'smart_187_rw10_std', 'smart_187_vel',
       'smart_188_from_mean_10', 'smart_188_raw', 'smart_188_rw10_mean',
       'smart_188_rw10_std', 'smart_188_vel','smart_197_from_mean_10', 'smart_197_raw', 'smart_197_rw10_mean',
       'smart_197_rw10_std', 'smart_197_vel',
       'smart_198_from_mean_10', 'smart_198_raw', 'smart_198_rw10_mean',
       'smart_198_rw10_std', 'smart_198_vel','smart_5_from_mean_10',
       'smart_5_raw', 'smart_5_rw10_mean', 'smart_5_rw10_std','smart_5_vel']]

In [6]:
hard_drive_df.info(null_counts=True)

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 35709 entries, 0 to 35708
Data columns (total 31 columns):
date                      35709 non-null datetime64[ns]
remaining_useful_life     35709 non-null int64
RUL_30                    35709 non-null int64
failure                   35709 non-null int64
model                     35709 non-null object
serial_number             35709 non-null object
smart_187_from_mean_10    33199 non-null float64
smart_187_raw             35709 non-null float64
smart_187_rw10_mean       33199 non-null float64
smart_187_rw10_std        33199 non-null float64
smart_187_vel             35426 non-null float64
smart_188_from_mean_10    33199 non-null float64
smart_188_raw             35709 non-null float64
smart_188_rw10_mean       33199 non-null float64
smart_188_rw10_std        33199 non-null float64
smart_188_vel             35426 non-null float64
smart_197_from_mean_10    33199 non-null float64
smart_197_raw             35709 non-null float64
smart_197_

The hard drive data contains some nan values created in the process of creating new features (some will have 10 nans per hard drive info, some 20, some 30, etc). Depending on the features used we will remove the coresponding rows containing nans

In [7]:
hard_drive_df = hard_drive_df.dropna(how='any').reset_index()   
hard_drive_df = hard_drive_df.drop('index', axis=1)

In [8]:
hard_drive_df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 33199 entries, 0 to 33198
Data columns (total 31 columns):
date                      33199 non-null datetime64[ns]
remaining_useful_life     33199 non-null int64
RUL_30                    33199 non-null int64
failure                   33199 non-null int64
model                     33199 non-null object
serial_number             33199 non-null object
smart_187_from_mean_10    33199 non-null float64
smart_187_raw             33199 non-null float64
smart_187_rw10_mean       33199 non-null float64
smart_187_rw10_std        33199 non-null float64
smart_187_vel             33199 non-null float64
smart_188_from_mean_10    33199 non-null float64
smart_188_raw             33199 non-null float64
smart_188_rw10_mean       33199 non-null float64
smart_188_rw10_std        33199 non-null float64
smart_188_vel             33199 non-null float64
smart_197_from_mean_10    33199 non-null float64
smart_197_raw             33199 non-null float64
smart_197_

In [9]:
len(hard_drive_df[hard_drive_df.failure == 1].serial_number.unique())

277

In [10]:
hard_drive_df.columns

Index(['date', 'remaining_useful_life', 'RUL_30', 'failure', 'model',
       'serial_number', 'smart_187_from_mean_10', 'smart_187_raw',
       'smart_187_rw10_mean', 'smart_187_rw10_std', 'smart_187_vel',
       'smart_188_from_mean_10', 'smart_188_raw', 'smart_188_rw10_mean',
       'smart_188_rw10_std', 'smart_188_vel', 'smart_197_from_mean_10',
       'smart_197_raw', 'smart_197_rw10_mean', 'smart_197_rw10_std',
       'smart_197_vel', 'smart_198_from_mean_10', 'smart_198_raw',
       'smart_198_rw10_mean', 'smart_198_rw10_std', 'smart_198_vel',
       'smart_5_from_mean_10', 'smart_5_raw', 'smart_5_rw10_mean',
       'smart_5_rw10_std', 'smart_5_vel'],
      dtype='object')

In [123]:
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestRegressor,GradientBoostingRegressor
from sklearn.linear_model import LinearRegression, Ridge, Lasso, ElasticNet,SGDRegressor
from sklearn.preprocessing import StandardScaler
from sklearn.metrics import mean_squared_error, r2_score
import statsmodels.api as sm
from sklearn.neighbors import KNeighborsRegressor
from imblearn.over_sampling import RandomOverSampler

Using TensorFlow backend.
/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:526: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint8 = np.dtype([("qint8", np.int8, 1)])
/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:527: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_quint8 = np.dtype([("quint8", np.uint8, 1)])
/opt/anaconda3/envs/metis/lib/python3.7/site-packages/tensorflow/python/framework/dtypes.py:528: FutureWarning: Passing (type, 1) or '1type' as a synonym of type is deprecated; in a future version of numpy, it will be understood as (type, (1,)) / '(1,)type'.
  _np_qint16 = np.dtype([("qint16", np.int16, 1)])
/opt/anaconda3/envs/metis/lib/python3.7/site-pa

In [29]:
# 25 % test and 75 % train
failed_hard_drive_list = hard_drive_df[hard_drive_df.failure == 1].serial_number.unique()

In [30]:
# split the failed and working hard drive into 25% and 75% to use for train and test set
train_hd_failed, test_hd_failed = train_test_split(failed_hard_drive_list,test_size=0.25, random_state=42)


In [60]:
# get the train and test dataframes
train_df = hard_drive_df[hard_drive_df.serial_number.isin(train_hd_failed)]
test_df = hard_drive_df[hard_drive_df.serial_number.isin(test_hd_failed)]

In [104]:
train_df.sample(frac=1)
test_df.sample(frac=1)


,date,remaining_useful_life,RUL_30,failure,model,serial_number,smart_187_from_mean_10,smart_187_raw,smart_187_rw10_mean,smart_187_rw10_std,...,smart_198_from_mean_10,smart_198_raw,smart_198_rw10_mean,smart_198_rw10_std,smart_198_vel,smart_5_from_mean_10,smart_5_raw,smart_5_rw10_mean,smart_5_rw10_std,smart_5_vel
30961,2019-04-07,107,42,0,ST4000DM000,Z305D66M,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
18645,2019-04-25,21,21,0,ST4000DM000,Z30468K9,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
8309,2019-06-12,105,42,0,ST4000DM000,S301M4YT,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
8236,2019-03-31,178,42,0,ST4000DM000,S301M4YT,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
1302,2019-03-19,15,15,0,ST4000DM000,S300Z4YH,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
33069,2019-07-20,39,42,0,ST4000DM000,Z305L695,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
9712,2019-03-12,136,42,0,ST4000DM000,S301PQ1F,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
7953,2019-05-14,98,42,0,ST4000DM000,S301KWTE,0.0,0.0,0.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0
27895,2019-01-12,43,42,0,ST4000DM000,Z30513PV,0.0,8.0,8.0,0.0,...,0.0,0.0,0.0,0.000000e+00,0.0,0.0,0.0,0.0,0.0,0.0


In [65]:
train_df['remaining_useful_life'].std()

59.83741884289632

In [33]:
# split the train into 3 groups (3 fold)
kfold_5 = np.array_split(train_hd_failed,3)

In [71]:
mask = ['RUL_30','smart_5_raw','smart_187_raw','smart_188_raw','smart_197_raw','smart_198_raw']


In [72]:
train_df.columns

Index(['date', 'remaining_useful_life', 'RUL_30', 'failure', 'model',
       'serial_number', 'smart_187_from_mean_10', 'smart_187_raw',
       'smart_187_rw10_mean', 'smart_187_rw10_std', 'smart_187_vel',
       'smart_188_from_mean_10', 'smart_188_raw', 'smart_188_rw10_mean',
       'smart_188_rw10_std', 'smart_188_vel', 'smart_197_from_mean_10',
       'smart_197_raw', 'smart_197_rw10_mean', 'smart_197_rw10_std',
       'smart_197_vel', 'smart_198_from_mean_10', 'smart_198_raw',
       'smart_198_rw10_mean', 'smart_198_rw10_std', 'smart_198_vel',
       'smart_5_from_mean_10', 'smart_5_raw', 'smart_5_rw10_mean',
       'smart_5_rw10_std', 'smart_5_vel'],
      dtype='object')

In [125]:
r2_score_list_tr = []
r2_score_list_vl = []
for val_group_index in range(len(kfold_5)):
    # get serial numbers in val and train sets
    train_s_nums = kfold_5[:val_group_index]+kfold_5[val_group_index+1:]
    val_s_nums = kfold_5[val_group_index]

    # get the data for train and val sets
    train_data = train_df[train_df.serial_number.isin(np.concatenate(train_s_nums))][mask]
    val_data = train_df[train_df.serial_number.isin(val_s_nums)][mask]

    train_data.sample(frac=1)
    val_data.sample(frac=1)
    
    # get X_tr,y_tr,X_val, y_val
    X_tr, y_tr = train_data[mask].drop('RUL_30',axis=1), train_data['RUL_30'] 
    X_vl, y_vl = val_data[mask].drop('RUL_30',axis=1), val_data['RUL_30'] 
    
    # scale the data
    #scaler = StandardScaler()
    #X_tr_scaled = scaler.fit_transform(X_tr)
    #X_vl_scaled = scaler.transform(X_vl)

    # over sample 
    ros = RandomOverSampler(random_state=1)
    X_resampled, y_resampled = ros.fit_sample(X_tr, y_tr)

    # under sample
    #ind = y_tr[y_tr==0].sample(len(y_tr[y_tr==1])).index
    #y_resampled = pd.concat([y_tr[ind],y_tr[y_tr==1]])
    #X_resampled = X_tr.loc[y_resampled.index]
    #y_resampled = y_tr
    #X_resampled = X_tr_scaled
    #X_vl = X_vl_scaled
    # model
    model = RandomForestRegressor(n_estimators=50)
    #model = SGDRegressor()
    model.fit(X_resampled,y_resampled)
    #model = sm.GLM(y_resampled, sm.add_constant(X_resampled), family=sm.families.Poisson())
    #poisson_training_results = model.fit()

    #train_predicted = (model.predict_proba(X_resampled)[:, 1] >= threshold)
    train_predicted = model.predict(X_resampled)
    #val_predicted = (model.predict_proba(X_vl)[:, 1] >= threshold)
    val_predicted = model.predict(X_vl)
    
    
    r2_score_list_tr.append(r2_score(y_resampled, train_predicted))
    
    r2_score_list_vl.append(r2_score(y_vl, val_predicted))

r2_tr_mean  = sum(r2_score_list_tr)/len(r2_score_list_tr)
r2_vl_mean = sum(r2_score_list_vl)/len(r2_score_list_vl)
print(r2_tr_mean)
print(r2_vl_mean)


0.18237256376357872
-2.7693110083883856


In [126]:
X_ts, y_ts = test_df[mask].drop('RUL_30',axis=1), test_df['RUL_30'] 

In [127]:
test_pred = model.predict(X_ts)

In [128]:
r2_score(y_ts,test_pred)

-2.8794527614932925

We get negative r2 values, whcih mean the models are doing really poorly. One of the reasons could be that the features do not change much overtime, do not degrade gradually, so it's hard to see the difference between 100 remaining useful life and 30. 